api 연결을 위한 설정

In [1]:
# PEP-8 컨벤션에 따르는게 좋다.
import os
import json
import time

import requests

In [3]:
class TMDBCrawler:
    def __init__( #던더 더블언더스코어
        self,
        region="KR",
        language="ko-KR",
        image_language="ko",
        request_interval_seconds=0.4
    ):
        self._base_url = os.environ.get("TMDB_BASE_URL", "https://api.themoviedb.org/3/movie")
        self._api_key = os.environ.get("TMDB_API_KEY")
        self._region = region
        self._language = language
        self._image_language = image_language
        self._request_interval_seconds = request_interval_seconds

    def get_popular_movies(self, page=1):
        params = {
            "api_key": self._api_key,
            "language": self._language,
            "region": self._region,
            "page": page
        }
        response = requests.get(f"{self._base_url}/popular", params=params)

        if response.status_code != 200:
            print(f"[ERROR] Failed to fetch page {page}: {response.status_code}")
            return []
        return response.json().get("results", [])

    def get_bulk_popular_movies(self, start_page=1, end_page=1):
        movies = []
        for page in range(start_page, end_page + 1):
            print(f"Fetching page {page}...")
            movies.extend(self.get_popular_movies(page))
            time.sleep(self._request_interval_seconds)
        return movies

    @staticmethod
    def save_movies_to_json_file(movies, dst="./result", filename="popular"):
        os.makedirs(dst, exist_ok=True)  
        data = {"movies": movies}
        
        file_path = os.path.join(dst, f"{filename}.json")
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"✅ 영화 데이터를 {file_path}에 저장 완료.")
